In [ ]:
import cv2
import numpy as np

In [ ]:


def getContour(img, thrshld, maxthrshld):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding
    _, binary_img = cv2.threshold(gray, thrshld, maxthrshld, cv2.THRESH_BINARY)

    # Find all contours (not just external ones)
    contours, _ = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out small contours (noise)
    min_area = 500  # Tune this based on your image
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # print(f"Total contours: {len(filtered_contours)}")
    
    # cv2.drawContours(img, filtered_contours, -1, (0,0,255),2)
    # cv2.imshow("multple contour", img)
    
    if not filtered_contours:
        return None

    # Sort contours by area
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea)

    # Assuming smallest relevant area is the inner ring
    inner_contour = sorted_contours[0]

    outer_contour = max(filtered_contours, key=cv2.contourArea)

    return inner_contour, outer_contour



In [2]:
def getCircleProperty(inner_contour, outer_contour):
    (center_xin, center_yin), inner_radius = cv2.minEnclosingCircle(inner_contour) #smallest enclosing circle for the  contour.
    inner_diameter = 2 * inner_radius
    print(f"Inner Diameter: {inner_diameter:.2f} pixels")

    (center_xout, center_yout), outer_radius = cv2.minEnclosingCircle(outer_contour)
    outer_diameter = 2 * outer_radius
    print(f"Outer Diameter: {outer_diameter: .2f} pixel")

    return inner_radius, outer_radius, center_xin, center_yin, center_xout, center_yout



In [ ]:
def Visual(img, inner_radius, outer_radius, center_xin, center_yin, center_xout, center_yout ):

    #inner circle red
    # cv2.putText()
    cv2.circle(img, (int(center_xin), int(center_yin)), int(inner_radius), (0, 0, 255), 2) 

    # #outer circle green
    cv2.circle(img, (int(center_xout), int(center_yout)), int(outer_radius), (0, 0, 255), 2) 

    # float to integers
    center = (int(center_xout), int(center_yout))
    radius = int(outer_radius)
    diameter = int(outer_radius)*2

    # Define the start and end points of the diameter line (horizontal)
    pt1 = (center[0] - radius, center[1])  # left side of circle
    pt2 = (center[0] + radius, center[1])  # right side of circle

    # Draw the diameter line in red
    cv2.line(img, pt1, pt2, (0, 0, 255), 2)

    # Put the diameter value near the center
    label = f"{diameter}px"
    cv2.putText(
        img,
        label,
        (center[0] - 40, center[1] - 10),  # offset above the line
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 0, 255),  # red text
        2,
        cv2.LINE_AA
    )
    cv2.imshow("Diameters", img)




In [4]:
import cv2

def draw_diameter_line(img, contour, color=(0, 0, 255), label_prefix="Diameter"):
    """
    Draws the diameter line of the minimum enclosing circle for the given contour,
    along with a label showing the diameter in pixels.

    Parameters:
        img (np.ndarray): Image on which to draw.
        contour (np.ndarray): The contour for which the diameter is drawn.
        color (tuple): BGR color for drawing. Default is red.
        label_prefix (str): Text prefix for the label (e.g., "Outer Diameter").

    Returns:
        img_with_diameter (np.ndarray): The image with diameter line and label drawn.
    """
    # Copy the image so the original is not modified
    img_with_diameter = img.copy()

    # Get minimum enclosing circle
    (center_x, center_y), radius = cv2.minEnclosingCircle(contour)
    center = (int(center_x), int(center_y))
    radius = int(radius)
    diameter = 2 * radius

    # Define the start and end of the diameter line (horizontal)
    pt1 = (center[0] - radius, center[1])  # left point
    pt2 = (center[0] + radius, center[1])  # right point

    # Draw the diameter line
    cv2.line(img_with_diameter, pt1, pt2, color, 2)

    # Draw the diameter label text
    label = f"{label_prefix}: {diameter}px"
    text_pos = (center[0] - 50, center[1] - 10)
    cv2.putText(
        img_with_diameter,
        label,
        text_pos,
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        color,
        2,
        cv2.LINE_AA
    )

    return img_with_diameter


In [5]:
from numpy import outer


img1 = cv2.imread("/home/dhvani/Subhiksha/or-face/or-face-dent/s1.png")
# img1 = img1[200:1000, 700:1600]
img1 = img1[200:1000, 300:1300]




# inner, outer, innerdia, outerdia, center_xin, center_yin, center_xout, center_yout = getContour(img1, 100,255)

# inner
# inner, outer, innerdia, center_xin, center_yin = getContour(img1, 100,255)

# outer
inner, outer = getContour(img1, 100,255)
inner_radius, outer_radius, center_xin, center_yin, center_xout, center_yout = getCircleProperty(inner, outer)

# outer = getODContour(img1)



#draw inner outer
if inner is not None:
    cv2.drawContours(img1, [inner], -1, (0, 255, 0), 2)
if outer is not None:
    cv2.drawContours(img1, [outer], -1, (0, 255, 0), 2)


cv2.imshow("Detected Inner and Outer Rings", img1)

#inner
# Visual(img1, innerdia, center_xin, center_yin)
imcopy = img1.copy()
#outer
# Visual(imcopy, inner_radius, outer_radius, center_xin, center_yin, center_xout, center_yout)



cv2.waitKey(0)
cv2.destroyAllWindows()


Inner Diameter: 328.89 pixels
Outer Diameter:  688.09 pixel


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [6]:
cv2.imshow("Outer ", img1)
diameter_outer = draw_diameter_line(img1, outer, color=(0, 0, 255), label_prefix="Diameter")
cv2.imshow("Outer ", img1)

In [7]:
diameter_outer.shape

(800, 1000, 3)

In [8]:
def get_outer_diameter_contour(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Find all contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    # Sort by area and assume the **largest circle** is the outer ring
    outer_contour = max(contours, key=cv2.contourArea)
    return outer_contour


In [9]:
a =[5]

def add(a):
    a.append(10)
    return a

In [10]:
add(a)
print(a)

[5, 10]


In [ ]:
a

[5, 10]

: 